# 1. Постановка задачи

Требуется защитить данные клиентов страховой компании «Хоть потоп». Разработать метод преобразования данных,такой чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Описание данных**

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.   
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

# 2. Загрузка данных 

## 2.1. Импорт библиотек и загрузка данных 

**Импорт библиотек**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

**Импорт данных**

In [2]:
def read_data_set(pth1, pth2) -> pd.DataFrame:
    df = pd.DataFrame    
    try:
        df = pd.read_csv(pth1)
    except OSError as e:
        df = pd.read_csv(pth2)
    return df

In [3]:
pth1 = '\Спринт 7. Линейная алгебра\datasets\insurance.csv'         
pth2 = 'https://code.s3.yandex.net/datasets/insurance.csv'

In [4]:
df = read_data_set(pth1, pth2);

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

In [7]:
df.sample(10)

,пол,возраст,зарплата,члены_семьи,страховые_выплаты
3724,1,35.0,42500.0,0,0
4948,1,37.0,38000.0,1,0
4162,0,46.0,37700.0,1,1
2352,1,45.0,42000.0,2,1
1700,0,37.0,30900.0,1,0
3874,0,23.0,37500.0,0,0
3031,1,47.0,54400.0,1,1
3208,0,42.0,33400.0,2,0
1270,0,35.0,46100.0,0,0
3439,1,27.0,30900.0,0,0


In [8]:
df.duplicated().sum()

153

In [9]:
df.isna().sum()

пол                  0
возраст              0
зарплата             0
члены_семьи          0
страховые_выплаты    0
dtype: int64

Пропусков нет. Тип переменной, в которой записан пол можно поменять на булевый, но для нас не критично

In [10]:
df.describe()

,пол,возраст,зарплата,члены_семьи,страховые_выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [11]:
df.value_counts()

пол  возраст  зарплата  члены_семьи  страховые_выплаты
1    19.0     43200.0   1            0                    3
0    21.0     37200.0   0            0                    3
1    30.0     49900.0   1            0                    3
0    28.0     39800.0   1            0                    3
     31.0     49400.0   1            0                    3
                                                         ..
     35.0     37400.0   0            0                    1
              37100.0   0            0                    1
              36200.0   1            0                    1
              36100.0   2            0                    1
1    65.0     39700.0   1            5                    1
Length: 4847, dtype: int64

# 2.  Исследования операций с линейной регрессией

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.) Обучим линейную регрессию и посмотрим 

In [12]:
model = LinearRegression()
random_matrix = np.random.randint(1, 10, (4, 4))
np.linalg.inv(random_matrix)
features = df.drop(columns= ['страховые_выплаты'])
target = df['страховые_выплаты']
features_train, features_test, target_train, target_test = (train_test_split(features, target, train_size=0.75, 
                                                                             random_state=12345))    

In [13]:
model.fit(features_train , target_train)

LinearRegression()

In [14]:
r2_score(target_test, model.predict(features_test))

0.43522757127026546

In [15]:
features = features @ random_matrix
features_train, features_test, target_train, target_test = (train_test_split(features, target, train_size=0.75, 
                                                                             random_state=12345))  

In [16]:
model.fit(features_train, target_train)

LinearRegression()

In [17]:
r2_score(target_test, model.predict(features_test))

0.4352275712699484

Результаты в пределах погрешности вычислений.  
Обоснование:


Xo - рандомная обратимая матрица
   
$$
w' = (X'^TX')^{-1}X'^Ty
$$
$$   
a' = X'w'
$$ 
$$   
X' = XXo
$$  
$$    
w' = ((XXo)^TXXo)^{-1}(XXo)^Ty = Xo^{-1}(X^TX)^{-1}(Xo^T)^{-1} Xo^T X^T y  = Xo^{-1}(X^TX)^{-1} X^T y 
$$ 
$$
w = (X^TX)^{-1}X^Ty
$$
$$
w' = Xo^{-1}w 
$$    
$$
a' = XXoXo^{-1}w = Xw = a
$$ 


# 3.  Алгоритм защиты данных и реализация алгоритма 

Так как качество работы не меняется, будем домножать матрицу признаков на случайную обратимую матрицу.
Для каждого датасета можно изспользовать разные матрицы. 

In [18]:
random_matrix = np.random.randint(1, 10, (4, 4))
features = df.drop(columns= ['страховые_выплаты'])
features = features @ random_matrix
features_train, features_test, target_train, target_test = (train_test_split(features, target, train_size=0.75, 
                                                                             random_state=12345)) 

In [19]:
np.linalg.inv(random_matrix)

array([[ 0.06060606,  0.34090909, -0.16666667, -0.16666667],
       [-0.04545455, -0.06818182,  0.5       , -0.5       ],
       [ 0.18181818,  0.02272727, -0.5       ,  0.5       ],
       [-0.10606061, -0.15909091,  0.16666667,  0.16666667]])

In [20]:
model.fit(features_train, target_train)
r2_score(target_test, model.predict(features_test))

0.4352275712702788

Обоснование дано в пункте 2

# 4.  Вывод

Постановка задачи:

Защитить данные клиентов страховой компании «Хоть потоп». Разработать метод преобразования данных,такой чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Работа была выполнена в несколько этапов:

    - Загрузка данных  
    - Исследования операций с линейной регрессией
    - Алгоритм защиты данных и реализация алгоритма 

На первом этапе были произведены загрузки и анализ данных


На втором этапе было дано обоснование причины по которой при перемножении признаков точность модели не изменяется

На третьем этапе был предложен алгоритм шифрования данных (умножение на случайную матрицу) и представлена реализация